# Контекстные менеджеры

Доклад Скотта Майерса "Why C++ Sails When the Vasa Sank"

- "What you would consider the single most important feature in C++?"
- Destructors. It is RAII (Resource Acquisition Is Initialization)

RAII:
- Получение ресурса - инициализация
- Освобождение ресурса - уничтожение

In [ ]:
with open('file.txt', 'w') as f:
    print('THIS IS RAII', file=f)

In [ ]:
import io


def raise_exception(f: io.TextIOWrapper):
    raise Exception('Some unexpected exception here')


def open_the_file_and_do_smth():
    f = open('file_and_do_smth.txt', 'w')
    try:
        raise_exception()
    except Exception:
        f.close()
        raise
    print('WRITE IT', file=f)
    f.close()
    
    
open_the_file_and_do_smth()

In [ ]:
import io


def raise_exception(f: io.TextIOWrapper):
    raise Exception('Some unexpected exception here')


def open_the_file_and_do_smth():
    with open('file_and_do_smth.txt', 'w') as f:
        raise_exception()
        print('WRITE IT', file=f)


open_the_file_and_do_smth()

In [ ]:
with open('file1.txt', 'w') as f:
    f.write('Hello')
# file is closed
f.write('world')

Примеры контекстных менеджеров для `warnings`

In [ ]:
import numpy as np
import warnings


with warnings.catch_warnings(record=True) as w:
    np.int32(1) / np.int32(0)
    np.log(0)
    
    for warn in w:
        print(warn)

`contextlib.contextmanager` — удобный способ создавать контекстные менеджеры

In [ ]:
from contextlib import contextmanager


@contextmanager
def first():
    print('before')
    yield 'first'
    print('after')
    
    
class First:
    def __enter__(self):
        print('before')
        return 'first'

    # * exc_type - ошибки, которые были пойманы за время работы
    # * exc_value - значения ошибок (помним, например, текст)
    # * exc_traceback - объект с тем, где это было (чаще всего не используется)
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('after')
    
    
@contextmanager
def second():
    print('before')
    yield 2
    print('after')
    

class Second:
    def __enter__(self):
        print('before')
        return 2
        
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('after')

In [ ]:
with first() as f:
    print(f)

print()
    
with First() as f:
    print(f)

Несколько контекстных менеджеров, вложенные контекстные менеджеры

In [ ]:
# nested contexts
with first() as f, second() as s:
    print(f, s)

In [ ]:
with first() as f:
    with second() as s:
        print(f, s)

`as smth` - опционально

In [ ]:
with first():
    pass

# Dataclass

In [3]:
import typing as tp
from dataclasses import dataclass


T = tp.TypeVar('T')


@dataclass
class Case:
    name: str
    result: T
    expected: T


c = Case("test1", 2, 1)

In [4]:
import typing as tp
from dataclasses import dataclass, field


T = tp.TypeVar('T')


# https://docs.python.org/3/library/dataclasses.html#dataclasses.dataclass
@dataclass(init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)
class Case:
    name: str
    result: T
    expected: T


c = Case("test_name", 2, 1)

In [5]:
from enum import Enum
from dataclasses import dataclass


class Marks(Enum):
    Two = 2
    Three = 3
    Four = 4
    Five = 5
    
    
class Abilities(Enum):
    Smart = 0
    Strong = 1
    Funny = 2
    Etc = 3
    
    
@dataclass
class Student:
    first_name: str
    second_name: str
    marks: list[Marks]
    abilities: tuple[Abilities, ...]

In [6]:
from dataclasses import (
    dataclass,
    field,
)
from abc import (
    ABC,
    abstractmethod,
)


DISCOUNT_PERCENTS = 15


@dataclass(frozen=True, order=True)
class Item:
    item_id: int = field(compare=False)
    title: str
    cost: int

    def __post_init__(self) -> None:
        assert len(self.title) != 0 and self.cost > 0
        
        
@dataclass    # type: ignore
class Position(ABC):
    item: Item

    @abstractmethod
    def cost(self) -> float:
        return 0.


@dataclass
class Order:
    order_id: int
    positions: list[Position] = field(default_factory=list)
    cost: int = field(init=False)
    have_promo: bool = False

    def __post_init__(self, have_promo: bool) -> None:
        pos_sum: float = sum([getattr(p, 'cost') for p in self.positions])
        if have_promo:
            self.cost = int(pos_sum / 100 * (100 - DISCOUNT_PERCENTS))
        else:
            self.cost = int(pos_sum)
